In [1]:
import datetime as dt
import pandas as pd
from calval_data_access.mark_data import get_db_data, get_user_db_creds, query_constructor
from calval_data_access.reference_data import get_processed_data

In [2]:
treatment_by_device = {"D003898":"BV2_L1", "D003960":"BV2_L2", "D003942":"BV2_H1", "D003943":"BV2_H2",
"D003701":"GB_L1", "D003705":"GB_L2", "D003978":"GB_H2"} #GB_H1 excluded because Mark device was switched out
start_date = dt.datetime(2023,5,1)
end_date = dt.datetime(2023,8,31)

# get mark data

In [3]:
creds = get_user_db_creds(user = "jacob_goldberg", environment = "alp")

In [ ]:
data_by_device = {}
for device in treatment_by_device.keys():
    device_query = query_constructor("calibrated", device, start_date, end_date)
    device_data = get_db_data(creds, device_query)
    data_by_device[device] = device_data
    

# get reference data

In [ ]:
reference_data_by_device = {}
for device, treatment in treatment_by_device.items():
    site = "TWE_" + treatment.split("_")[0]
    source = treatment.split("_")[1]
    device_ref_data = get_processed_data(site, source, start_date, end_date)
    reference_data_by_device[device] = device_ref_data

# join data

In [ ]:
data_by_device["D003898"].head()

In [ ]:
reference_data_by_device["D003898"].head()

In [ ]:
joined_data_by_device = {}
for device, treatment in treatment_by_device.items():
    temp_device_data = data_by_device[device]
    temp_reference_data = reference_data_by_device[device]
    temp_reference_data.ref_time = pd.to_datetime(temp_reference_data.ref_time, utc = True)
    temp_joined_device_data = pd.merge(temp_device_data, temp_reference_data, how = "outer", left_on = "time", right_on = "ref_time") 
    temp_joined_device_data = temp_joined_device_data.filter(["time", "device", "tbelow", "ref_tbelow", "site_id", "source"])
    joined_data_by_device[device] = temp_joined_device_data

In [ ]:
joined_data_by_device[device].head()

In [ ]:
joined_data_by_device[device].columns

In [ ]:
joined_data_by_device[device].head()
